In [ ]:
from datasets import load_dataset
gtzan = load_dataset("thisisjibon/banglabeats", "all")
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan["train"][0]

id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

  0%|          | 0/1 [00:00<?, ?it/s]

'Indie'

In [ ]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [ ]:
from datasets import Audio
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/2af8452ed4cff3e02efc93bd9c55c02052463d3235a482097cea1b80c323bb44/wavs3sec/Indie/1599.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/2af8452ed4cff3e02efc93bd9c55c02052463d3235a482097cea1b80c323bb44/wavs3sec/Indie/1599.wav',
  'array': array([-0.0131646 , -0.02055288, -0.01401616, ..., -0.31587178,
         -0.31501222, -0.30590048], dtype=float32),
  'sampling_rate': 16000},
 'genre': 3}

In [ ]:
import numpy as np
sample = gtzan["train"][0]["audio"]
print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

In [ ]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])
print(f"inputs keys: {list(inputs.keys())}")
print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)


In [ ]:
from pydub import AudioSegment
def downmix_audio(audio_array):
    # Convert stereo audio to mono
    stereo_audio = AudioSegment.from_numpy(audio_array)
    mono_audio = stereo_audio.set_channels(1)
    return mono_audio.to_numpy()

In [ ]:
max_duration = 3.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

  0%|          | 0/146 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 14553
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 1617
    })
})

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}
id2label["7"]

'Rock'

In [ ]:
from transformers import AutoModelForAudioClassification
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.weight', 'projector.bias', 'classifier.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 2
num_train_epochs = 20

training_args = TrainingArguments(
    f"{model_name}-finetuned-banglabeats",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=6e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.02,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    seed=42,
    push_to_hub=True,
)


# from transformers import TrainingArguments

# model_name = model_id.split("/")[-1]
# batch_size = 4  # Change to 4 as specified
# gradient_accumulation_steps = 2
# num_train_epochs = 20  # Change to 18 as specified

# training_args = TrainingArguments(
#     f"{model_name}-finetuned-banglabeats",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=batch_size,
#     gradient_accumulation_steps=gradient_accumulation_steps,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=num_train_epochs,
#     warmup_ratio=0.1,
#     logging_steps=5,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     lr_scheduler_type="linear",
#     fp16=True,
#     push_to_hub=True,
#     save_total_limit=1,  # Limit the number of checkpoint saves to save disk space
# )


In [ ]:
! pip install -q gradio transformers datasets[audio] accelerate evaluate wandb

In [ ]:
import evaluate
import numpy as np
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)



In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.943900,0.927364,0.642548
2,0.854000,0.749769,0.726036
3,0.483500,0.632925,0.770563
4,0.622600,0.615927,0.793445
5,0.456000,0.711775,0.797155
6,0.056500,0.799352,0.805195
7,0.260500,0.973516,0.815090
8,0.363500,1.061817,0.824366
9,0.187900,1.164404,0.821274
10,0.029200,1.254317,0.819419


TrainOutput(global_step=18200, training_loss=0.2112818535941315, metrics={'train_runtime': 28584.2977, 'train_samples_per_second': 10.183, 'train_steps_per_second': 0.637, 'total_flos': 1.98589532004864e+18, 'train_loss': 0.2112818535941315, 'epoch': 20.0})

In [ ]:
kwargs = {
    "dataset_tags": "thisisjibon/banglabeats",
    "dataset": "BanglaBeats",
    "model_name": f"{model_name}-finetuned-banglabeats",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}
trainer.push_to_hub(**kwargs)

'https://huggingface.co/thisisjibon/distilhubert-finetuned-banglabeats/tree/main/'